In [ ]:
!pip install pytorch_lightning

In [6]:
import pandas as pd
import numpy as np

In [7]:
transactions = pd.read_csv("/content/drive/MyDrive/colab_data/kaggle_H&M/data/transactions_raw.csv")


In [8]:
len(transactions.customer_id.unique())

1362281

In [9]:
ratings = transactions[-8000000:]
# ratings = transactions[-1000000:]

In [10]:
ratings[-8000000:]

,t_dat,customer_id,article_id,price,sales_channel_id
23788324,2020-03-27,b4fb618ef7d51d9256adfa9ef51eb6045721559bf7ade3...,511105004,0.022017,2
23788325,2020-03-27,b4fc1fa268ece296304a8c361749d84dd21dd109dbacef...,829147002,0.042356,2
23788326,2020-03-27,b4fc1fa268ece296304a8c361749d84dd21dd109dbacef...,829147002,0.042356,2
23788327,2020-03-27,b4fc1fa268ece296304a8c361749d84dd21dd109dbacef...,797835006,0.067780,2
23788328,2020-03-27,b4ff232734321261557f09176c2aa3c2467702187c0da0...,854043007,0.027441,2
...,...,...,...,...,...
31788319,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,929511001,0.059305,2
31788320,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,891322004,0.042356,2
31788321,2020-09-22,fff380805474b287b05cb2a7507b9a013482f7dd0bce0e...,918325001,0.043203,1
31788322,2020-09-22,fff4d3a8b1f3b60af93e78c30a7cb4cf75edaf2590d3e5...,833459002,0.006763,1


In [11]:
ratings['rank_latest'] = ratings.groupby(['customer_id'])['t_dat'].rank(method = 'first', ascending = False)
# ratings = ratings.query(' rank_latest < 7')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [12]:
def map_column(df: pd.DataFrame, col_name: str):
    """
    Maps column values to integers
    :param df:
    :param col_name:
    :return:
    """
    values = sorted(list(df[col_name].unique()))
    mapping = {k: i + 1 for i, k in enumerate(values)}
    inverse_mapping = {v: k for k, v in mapping.items()}

    df[col_name + "_mapped"] = df[col_name].map(mapping)

    return df, mapping, inverse_mapping

ratings, cus_mapping, cus_inverse_mapping = map_column(ratings, col_name="customer_id")
ratings, art_mapping, art_inverse_mapping = map_column(ratings, col_name="article_id")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [13]:

val_start_date = '2020-09-16'
train_ratings = ratings.query(f"t_dat < '{val_start_date}'").reset_index(drop=True)
valid_ratings = ratings.query(f"t_dat >= '{val_start_date}'").reset_index(drop=True)

In [14]:
train_ratings

,t_dat,customer_id,article_id,price,sales_channel_id,rank_latest,customer_id_mapped,article_id_mapped
0,2020-03-27,b4fb618ef7d51d9256adfa9ef51eb6045721559bf7ade3...,511105004,0.022017,2,6.0,522869,2171
1,2020-03-27,b4fc1fa268ece296304a8c361749d84dd21dd109dbacef...,829147002,0.042356,2,22.0,522882,34431
2,2020-03-27,b4fc1fa268ece296304a8c361749d84dd21dd109dbacef...,829147002,0.042356,2,23.0,522882,34431
3,2020-03-27,b4fc1fa268ece296304a8c361749d84dd21dd109dbacef...,797835006,0.067780,2,24.0,522882,27525
4,2020-03-27,b4ff232734321261557f09176c2aa3c2467702187c0da0...,854043007,0.027441,2,2.0,522921,38899
...,...,...,...,...,...,...,...,...
7759684,2020-09-15,ffe41634ff990908faacbb465063e027e7c39499f8dfc1...,850917001,0.025407,1,3.0,739470,38152
7759685,2020-09-15,ffef8aec5cf011fa1393b40337a5993ce0b7b81af6b322...,853316001,0.008458,1,4.0,739598,38692
7759686,2020-09-15,ffef8aec5cf011fa1393b40337a5993ce0b7b81af6b322...,296366006,0.000847,1,5.0,739598,342
7759687,2020-09-15,ffef8aec5cf011fa1393b40337a5993ce0b7b81af6b322...,789769001,0.013542,1,6.0,739598,26169


In [15]:
import torch
from torch.utils.data import Dataset

class MovieLensTrainDataset(Dataset):
    """MovieLens PyTorch Dataset for Training
    
    Args:
        ratings (pd.DataFrame): Dataframe containing the movie ratings
        all_movieIds (list): List containing all movieIds
    
    """

    def __init__(self, ratings, all_movieIds):
        self.users, self.items, self.labels = self.get_dataset(ratings, all_movieIds)

    def __len__(self):
        return len(self.users)
  
    def __getitem__(self, idx):
        return self.users[idx], self.items[idx], self.labels[idx]

    def get_dataset(self, ratings, all_movieIds):
        users, items, labels = [], [], []
        user_item_set = set(zip(ratings['customer_id_mapped'], ratings['article_id_mapped']))

        num_negatives = 5
        for u, i in user_item_set:
            users.append(u)
            items.append(i)
            labels.append(1)
            for _ in range(num_negatives):
                negative_item = np.random.choice(all_movieIds)
                while (u, negative_item) in user_item_set:
                    negative_item = np.random.choice(all_movieIds)
                users.append(u)
                items.append(negative_item)
                labels.append(0)

        return torch.tensor(users), torch.tensor(items), torch.tensor(labels)


In [16]:
import torch.nn as nn
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from pytorch_lightning.callbacks import ModelCheckpoint

class NCF(pl.LightningModule):
    """ Neural Collaborative Filtering (NCF)
    
        Args:
            num_users (int): Number of unique users
            num_items (int): Number of unique items
            ratings (pd.DataFrame): Dataframe containing the movie ratings for training
            all_movieIds (list): List containing all movieIds (train + test)
    """
    
    def __init__(self, num_users, num_items, ratings, all_movieIds):
        super().__init__()
        self.user_embedding = nn.Embedding(num_embeddings=num_users, embedding_dim=8)
        self.item_embedding = nn.Embedding(num_embeddings=num_items, embedding_dim=8)
        self.fc1 = nn.Linear(in_features=16, out_features=64)
        self.fc2 = nn.Linear(in_features=64, out_features=32)
        self.output = nn.Linear(in_features=32, out_features=1)
        self.ratings = ratings
        self.all_movieIds = all_movieIds
        
    def forward(self, user_input, item_input):
        
        # Pass through embedding layers
        user_embedded = self.user_embedding(user_input)
        item_embedded = self.item_embedding(item_input)

        # Concat the two embedding layers
        vector = torch.cat([user_embedded, item_embedded], dim=-1)

        # Pass through dense layer
        vector = nn.ReLU()(self.fc1(vector))
        vector = nn.ReLU()(self.fc2(vector))

        # Output layer
        pred = nn.Sigmoid()(self.output(vector))

        return pred
    
    def training_step(self, batch, batch_idx):
        user_input, item_input, labels = batch
        predicted_labels = self(user_input, item_input)
        loss = nn.BCELoss()(predicted_labels, labels.view(-1, 1).float())
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters())

    def train_dataloader(self):
        return DataLoader(MovieLensTrainDataset(self.ratings, self.all_movieIds),
                          batch_size=256, num_workers=4)

In [ ]:
from pytorch_lightning.callbacks.base import Callback
num_users = len(ratings['customer_id_mapped'].unique())+1
num_items = len(ratings['article_id_mapped'].unique())+1
all_movieIds = ratings['article_id_mapped'].unique()

model_dir = "/content/drive/MyDrive/colab_data/kaggle_H&M/" + "saved_model/recommender_models"

checkpoint_callback = ModelCheckpoint(
    monitor="valid_loss",
    mode="min",
    dirpath=model_dir,
    filename="recommender",
)

model = NCF(num_users, num_items, train_ratings, all_movieIds)

trainer = pl.Trainer(
    max_epochs=15,
    gpus=1, 
    reload_dataloaders_every_epoch=True,
    progress_bar_refresh_rate=50, 
    logger=False, 
    # checkpoint_callback=True
    callbacks=[checkpoint_callback]
    )

trainer.fit(model)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:91: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=50)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  f"Setting `Trainer(progress_bar_refresh_rate={progress_bar_refresh_rate})` is deprecated in v1.5 and"
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:89: LightningDeprecationWarning: `reload_dataloaders_every_epoch` is deprecated in v1.4 and will be removed in v1.6. Please use `reload_dataloaders_every_n_epochs` in Trainer.
  "`reload_dataloaders_every_epoch` is deprecated in v1.4 and will be removed in v1.6."
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU avail

Training: 0it [00:00, ?it/s]

In [ ]:
torch.save(model, '/content/drive/MyDrive/colab_data/kaggle_H&M/saved_model/recommender_models/ncf_neg4_epoch3_batch216.ckpt')

In [ ]:
## Test Section